In [ ]:
# default_exp model.dcgan

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from fastcore.test import *

# DCGAN

> Model for Training 2 and 3 dim GANs

<img src="img/dcgan.png" alt="Drawing" style="width: 600px;"/>
     
https://towardsdatascience.com/fake-face-generator-using-dcgan-model-ae9322ccfd65

In [ ]:
# export

# General Includes
import torch
from torch import nn, optim
from torch import autograd

# Personal includes
from deeptool.architecture import Decoder, Discriminator
from deeptool.utils import Tracker

In [ ]:
# export

class DCGAN(nn.Module):
    """
    Modification of the DCGAN-Paper https://arxiv.org/pdf/1511.06434.pdf for 3-Dimensional tasks in MR-Imaging
    oriented on: https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
    """

    def __init__(self, device, args):
        """
        Setup the general architecture for the DCGAN model, composed of:
        Generator, Discriminator
        """
        super(DCGAN, self).__init__()
        # dimension of networks (2d conv or 3d conv)
        self.dim = args.dim

        # Number of channels depending on perspectives
        self.device = device
        self.n_chan = len(args.perspectives)

        # Generator
        self.generator = Decoder(args).to(self.device)
        

        # Encoding dimension
        self.n_z = args.n_z
        self.batch_size = args.batch_size

        # Fixed noise to visualize progression
        self.fixed_noise = torch.randn(
            self.batch_size, self.n_z, device=self.device)

        # lambda factor for gradient penatly
        self.lam = args.lam

        # Loss to be optimized for dcgan
        self.loss = nn.BCELoss()

        self.real_label = 1
        self.fake_label = 0

        if args.wgan == True:
            self.name = "wgan"
            self.forward = self.forward_wgan     
            # Discriminator
            self.discriminator = Discriminator(args, wgan=True).to(self.device)
            
        else:
            self.name = "dcgan"
            self.forward = self.forward_dcgan
            # Discriminator
            self.discriminator = Discriminator(args, wgan=False).to(self.device)

        # Optimizers
        self.optimizerGen = optim.Adam(
            self.generator.parameters(), lr=args.lr, betas=(0.5, 0.999))
        self.optimizerDis = optim.Adam(
            self.discriminator.parameters(), lr=args.lr, betas=(0.5, 0.999))

        # Setup the tracker to visualize the progress
        if args.track:
            self.tracker = Tracker(args, log_view=False)
        

    def watch_progress(self, test_data, iteration):
        """
        Outsourced to Tracker
        """
        self.tracker.track_progress(self, test_data, iteration)

    def calc_gradient_penalty(self, real_data, fake_data):
        """
        Apply the gradient Penalty for Discriminator training
        This is responsible for ensuring the Lipschitz constraint,
        which is required to ensure the Wasserstein distance.
        modified from: https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py 
        """
        # Asssign random factor alpha between 0 and 1
        sh = real_data.shape
        b_size = sh[0]
        alpha = torch.rand(b_size, 1)
        alpha = alpha.expand(b_size, int(
            real_data.nelement()/b_size)).contiguous().view(sh)
        alpha = alpha.to(self.device)

        # interpolating as disc input
        interpolates = (alpha * real_data +
                        ((1 - alpha) * fake_data)).to(self.device)
        interpolates = autograd.Variable(interpolates, requires_grad=True)

        # evaluate discriminator
        disc_interpolates = self.discriminator(interpolates)

        # calculate gradients
        gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                                  grad_outputs=torch.ones(
                                      disc_interpolates.size()).to(self.device),
                                  create_graph=True, retain_graph=True, only_inputs=True)[0]
        gradients = gradients.view(gradients.size(0), -1)

        # constrain gradients
        gradient_penalty = ((gradients.norm(2, dim=1) - 1)
                            ** 2).mean() * self.lam

        return gradient_penalty

    def sample_noise(self, batch_size, update):
        """
        Sample the latent noise required for training
        """
        if update:
            return torch.randn(batch_size, self.n_z, device=self.device)
        return self.fixed_noise

    def forward_wgan(self, data, update=True):
        """
        Calculate output and update networks with wgan
        """
        # get the image data
        real_gpu = data["img"].to(self.device)

        # (1) Update D network: maximize D(x) - D(G(z))
        # 1.1 Train with all-real batch
        self.discriminator.zero_grad()
        b_size = real_gpu.size(0)
        output = self.discriminator(real_gpu).view(-1)
        errD_real = -torch.mean(output)

        # 1.2 Train with all-fake batch
        noise = self.sample_noise(b_size, update)
        fake = self.generator(noise)
        output = self.discriminator(fake.detach()).view(-1)
        errD_fake = torch.mean(output)

        # 1.3 assign Gradient penalty
        gradient_penalty = self.calc_gradient_penalty(real_gpu, fake.detach())

        # sum the losses up
        errD = errD_fake + errD_real + gradient_penalty

        # Update Discriminator
        if update:
            errD.backward()
            self.optimizerDis.step()

        # (2) Update G network: maximize D(G(z))
        self.generator.zero_grad()
        output = self.discriminator(fake).view(-1)
        errG = -torch.mean(output)

        # Update Generator
        if update:
            errG.backward()
            self.optimizerGen.step()
            return fake

        else:
            # Track all relevant losses
            tr_data = {}
            tr_data["errD"] = errD.item()
            tr_data["errG"] = errG.item()
            tr_data["D_x"] = errD_real.item()
            tr_data["D_G_z1"] = errD_fake.item()
            tr_data["D_G_z2"] = output.mean().item()
            # Return losses and fake data
            return fake, tr_data

    def forward_dcgan(self, data, update=True):
        """
        Calculate output and update networks with dcgan
        """
        # get the image data
        real_gpu = data["img"].to(self.device)

        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        # 1.1 Train with all-real batch
        self.discriminator.zero_grad()
        b_size = real_gpu.size(0)
        label = torch.full((b_size,), self.real_label, device=self.device)
        output = self.discriminator(real_gpu).view(-1)
        errD_real = self.loss(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # 1.2 Train with all-fake batch
        noise = self.sample_noise(b_size, update)
        fake = self.generator(noise)
        output = self.discriminator(fake.detach()).view(-1)
        label.fill_(self.fake_label)
        errD_fake = self.loss(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_fake.item() + errD_real.item()

        # Update Discriminator
        if update:
            self.optimizerDis.step()

        # (2) Update G network: maximize 1 - log(D(G(z)))
        self.generator.zero_grad()
        label.fill_(self.real_label)
        output = self.discriminator(fake).view(-1)
        errG = self.loss(output, label)
        errG.backward()

        # Update Generator
        if update:
            self.optimizerGen.step()
            return fake

        else:
            # Track all relevant losses
            tr_data = {}
            tr_data["errD"] = errD
            tr_data["errG"] = errG.item()
            tr_data["D_x"] = D_x
            tr_data["D_G_z1"] = D_G_z1
            tr_data["D_G_z2"] = output.mean().item()
            # Return losses and fake data
            return fake, tr_data

In [ ]:
# hide
from deeptool.train_loop import test_one_batch
from deeptool.parameters import get_all_args, compat_args
args = get_all_args()
args.pic_size = 32

In [ ]:
# 3 dim test
args.model_type = "dcgan"
args.dim = 3
args = compat_args(args)
test_one_batch(args)

In [ ]:
# 2 dim test
args.dim = 2
args = compat_args(args)
test_one_batch(args)

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_rnn_vae.ipynb.
Converted 99_index.ipynb.
